In [4]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from glob import glob

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3281192208606192251
xla_global_id: -1
]


In [4]:
df_train = pd.read_csv("../data/train_labels.csv")
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}
df_train.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [7]:
labeled_files = glob('../data/train/*.tif')
test_files = glob('../data/test/*.tif')

In [8]:
print("labeled_files size :", len(labeled_files))
print("test_files size :", len(test_files))

labeled_files size : 220025
test_files size : 57458


In [12]:
train, val = train_test_split(df_train, test_size=0.1, random_state=42)

In [29]:
train_X, train_y = list(), list()
valid_X, valid_y = list(), list()

In [30]:
for idx, row in train.iterrows():
    image_path = os.path.join("../data/train/", row['id'] + ".tif")
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    train_X.append(image)
    train_y.append(row['label'])

train_X = np.array(train_X)
train_y = np.array(train_y)

In [ ]:
for idx, row in val.iterrows():
    image_path = os.path.join("../data/train/", row['id'] + ".tif")
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    
    valid_X.append(image)
    valid_y.append(row['label'])


valid_X = np.array(valid_X)
valid_y = np.array(valid_y)

In [ ]:
train_X[0].shape

array([None, None, None, ..., None, None, None], dtype=object)

In [16]:
def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

model = create_model(input_shape=(imagen_width, imagen_height, imagen_channels))
model.summary()

NameError: name 'imagen_width' is not defined

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_y, epochs=100, validation_data=(valid_X, valid_y), callbacks=[early_stopping])


In [ ]:
model.save('../models/trained_model.h5')

In [ ]:
path_test = "../data/test/"

test_files = [file for file in os.listdir(path_test) if file.endswith('.tif')]

test_X = []
file_names = []

for file in test_files:
    path_img = os.path.join(path_test, file)
    image = cv2.imread(path_img, cv2.IMREAD_UNCHANGED) 

    test_X.append(image)
    file_names.append(os.path.splitext(file)[0]) 

X_test = np.array(test_X)

preds = model.predict(X_test)

predicted_labels = (preds > 0.5).astype(int)

df_resultado = pd.DataFrame({'id': file_names, 'label': predicted_labels.flatten()})

print(df_resultado)

In [ ]:
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline_nasnet.csv", index=False)
df.head()